# Обучение модели классификации комментариев  

###Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию.

#### Нужно обучить модель классифицировать комментарии на позитивные и негативные.

Требование: метрика качества F1 должна быть не меньше 0.75 .

В нашем распоряжении данные из файла: `toxic_comments.csv`.

## Загрузка библеотек:

In [ ]:
## Для работы с данными:
# Импортируем pandas как pd:
import pandas as pd
# Импортируемnumpy как np:
import numpy as np
# Для упрощения работы используется кастомизированный класс Dataset:
from torch.utils.data import Dataset
# Импорт метода для создания выборок:
from sklearn.model_selection import train_test_split

# Импорт модели Fasttext:
from gensim.models import FastText
!pip install fasttext
import fasttext

# Импортируем токинайзер и модель(Bert):
import sklearn
import torch
!pip install transformers sentencepiece
from transformers import BertTokenizer
from transformers import BertForSequenceClassification

# Импорт "хэлперов":
from torch.utils.data import DataLoader
from transformers import AdamW
from transformers import get_linear_schedule_with_warmup

## Для оценки:
import math
# Импорт метрики:
from sklearn.metrics import precision_recall_fscore_support

# Библиотеки для подготовки текста:
import re
from gensim.parsing.preprocessing import STOPWORDS
from gensim.parsing.preprocessing import remove_stopwords

# Для загрузки файлов(у меня):
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


## Загрузка данных:

In [ ]:
try:
  df = pd.read_csv('/datasets/toxic_comments.csv', index_col=[0], parse_dates=[0])
except:
  df = pd.read_csv("/content/drive/MyDrive/For_data/toxic_comments.csv", index_col=[0], parse_dates=[0])

In [ ]:
df.sort_index(inplace=True)

In [ ]:
# Создание функции для изучения данных:
def information(data):
    print('Общая информация о таблице:')
    print(data.info(), '\n')
    print('Индекс монотонный? - ',data.index.is_monotonic)
    print('\n')
    print('Размер таблицы равен:', data.shape, '\n')
    print('Кол-во пропусков:', data.isna().sum(), '\n')
    print('Кол-во явных дубликатов:', data.duplicated().sum(), '\n')
    display(data.head())

In [ ]:
# Изучение информации о таблице:
information(df)

Общая информация о таблице:
<class 'pandas.core.frame.DataFrame'>
Int64Index: 159292 entries, 0 to 159450
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 3.6+ MB
None 

Индекс монотонный? -  True


Размер таблицы равен: (159292, 2) 

Кол-во пропусков: text     0
toxic    0
dtype: int64 



<ipython-input-18-b3ed42aba740>:5: FutureWarning: is_monotonic is deprecated and will be removed in a future version. Use is_monotonic_increasing instead.
  print('Индекс монотонный? - ',data.index.is_monotonic)


Кол-во явных дубликатов: 0 



,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


### Вывод:
У нас есть таблица размером 159292 строк.

Целевой признак находится в столбце toxic.

Явных дубликатов, как и пропусков нет.


# Fasttext:

## Подготовка данных:

#### Предварительная обработка текста

In [ ]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^\sa-zA-Z0-9@\[\]]',' ',text) # Удаляет пунктцацию
    text = re.sub(r'\w*\d+\w*', '', text) # Удаляет цифры
    text = re.sub('\s{2,}', " ", text) # Удаляет ненужные пробелы
    return text


In [ ]:
df['text'] = df['text'].apply(clean_text)

Разбитие данных на выборки:

In [ ]:
train, test = train_test_split(df,
                                   shuffle=False, test_size=0.2,train_size=0.8)

In [ ]:
with open('train.txt', 'w') as f:
    for each_text, each_label in zip(train['text'], train['toxic']):
        f.writelines(f'__label__{each_label} {each_text}\n')

with open('test.txt', 'w') as f:
    for each_text, each_label in zip(test['text'], test['toxic']):
        f.writelines(f'__label__{each_label} {each_text}\n')

## Работа с моделью:

In [ ]:
# Обучение модели
model6 = fasttext.train_supervised('train.txt',
                                   autotuneValidationFile='test.txt',
                                   autotuneMetric="f1:__label__1")


In [ ]:
# Создадим функцую для отображения результатов обучения модели
def print_results(sample_size, precision, recall):
    precision   = round(precision, 6)
    recall      = round(recall, 6)
    F1 = 2 * (precision * recall) / (precision + recall)

    print(f'{sample_size=}')
    print(f'{precision=}')
    print(f'{recall=}')
    print(f'{F1=}')

print_results(*model6.test('test.txt'))

sample_size=31859
precision=0.95885
recall=0.95885
F1=0.95885


# BERT:

## Подготовка данных:

In [ ]:
# Создание обучающей, тестовой и валидационной выборки:
train_val, test = train_test_split(df,
                                   shuffle=False, test_size=0.2,train_size=0.8)
train, val = train_test_split(train_val, test_size = 0.25,train_size =0.75)

#### Для упрощения работы используется кастомизированный класс Dataset:

In [ ]:
class CustomDataset(Dataset):

  # Метод, в котором мы инициализируем
  # тексты, метки, максимальную дину текста в токенах, а так же токенайзер:
  def __init__(self, texts, targets, tokenizer, max_len=512):
    self.texts = texts
    self.targets = targets
    self.tokenizer = tokenizer
    self.max_len = max_len

  # Метод len возвращает длину нашего датасета:
  def __len__(self):
    return len(self.texts)

  # Метод getitem возвращает словарь,
  # который состоит из самого исходного текста,
  # списка токенов, маски внимания, а также метки класса.
  def __getitem__(self, idx):
    text = str(self.texts[idx])
    target = self.targets[idx]

    encoding = self.tokenizer.encode_plus(
        text,
    # Исходный текст нужно обрамлять служебными токенами:
        add_special_tokens=True,
        max_length=self.max_len,
        return_token_type_ids=False,
    # Дополнять полученные векторы до максимальной длины :
        padding='max_length',
        return_attention_mask=True,
        return_tensors='pt',
        truncation=True
    )

    return {
      'text': text,
      'input_ids': encoding['input_ids'].flatten(),
      'attention_mask': encoding['attention_mask'].flatten(),
      'targets': torch.tensor(target, dtype=torch.long)
    }

## Модель:

In [ ]:
class BertClassifier:

    # Создание функции инициализации модели:
    def __init__(self, model_path, tokenizer_path, n_classes=2, epochs=1, model_save_path='/content/bert.pt'):
        self.model = BertForSequenceClassification.from_pretrained(model_path)
        self.tokenizer = BertTokenizer.from_pretrained(tokenizer_path)
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.model_save_path=model_save_path
        self.max_len = 512
        self.epochs = epochs
        self.out_features = self.model.bert.encoder.layer[1].output.dense.out_features
        self.model.classifier = torch.nn.Linear(self.out_features, n_classes)
        self.model.to(self.device)

    # Создание функции инициализации "хэлперов"
    #(помогают в обучении и оптимизации этого обучения):
    def preparation(self, X_train, y_train, X_valid, y_valid):
        # Создание datasets:
        self.train_set = CustomDataset(X_train, y_train, self.tokenizer)
        self.valid_set = CustomDataset(X_valid, y_valid, self.tokenizer)

        # Создание data loaders:
        self.train_loader = DataLoader(self.train_set, batch_size=2, shuffle=True)
        self.valid_loader = DataLoader(self.valid_set, batch_size=2, shuffle=True)

        # Инициализация помощников:
        self.optimizer = AdamW(self.model.parameters(), lr=2e-5, correct_bias=False)
        self.scheduler = get_linear_schedule_with_warmup(
                self.optimizer,
                num_warmup_steps=0,
                num_training_steps=len(self.train_loader) * self.epochs
            )
        self.loss_fn = torch.nn.CrossEntropyLoss().to(self.device)

    # Создание функции обучения модели в одной эпохе:
    def fit(self):
        self.model = self.model.train()
        losses = []
        correct_predictions = 0

        # Данные в цикле батчами генерируются с помощью DataLoader:
        for data in self.train_loader:
            input_ids = data["input_ids"].to(self.device)
            attention_mask = data["attention_mask"].to(self.device)
            targets = data["targets"].to(self.device)

            # Батч подается в модель:
            outputs = self.model(
                input_ids=input_ids,
                attention_mask=attention_mask
                )

            # На выходе получаем распределение вероятности
            # по классам и значение ошибки:
            preds = torch.argmax(outputs.logits, dim=1)
            loss = self.loss_fn(outputs.logits, targets)

            correct_predictions += torch.sum(preds == targets)

            losses.append(loss.item())

            # Делаем шаг на всех вспомогательных функциях:

            # Обратное распространение ошибки:
            loss.backward()
            # Jбрезаем градиенты для предотвращения "взрыва" градиентов:
            torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=1.0)
            # Шаг оптимизатора:
            self.optimizer.step()
            # Шаг планировщика:
            self.scheduler.step()
            # Обнуляем градиенты:
            self.optimizer.zero_grad()

        train_acc = correct_predictions.double() / len(self.train_set)
        train_loss = np.mean(losses)
        return train_acc, train_loss

    # Создание функции "оценки"(evaluation):
    def eval(self):
        self.model = self.model.eval()
        losses = []
        correct_predictions = 0

        with torch.no_grad():
            for data in self.valid_loader:
                input_ids = data["input_ids"].to(self.device)
                attention_mask = data["attention_mask"].to(self.device)
                targets = data["targets"].to(self.device)

                outputs = self.model(
                    input_ids=input_ids,
                    attention_mask=attention_mask
                    )

                preds = torch.argmax(outputs.logits, dim=1)
                loss = self.loss_fn(outputs.logits, targets)
                correct_predictions += torch.sum(preds == targets)
                losses.append(loss.item())

        val_acc = correct_predictions.double() / len(self.valid_set)
        val_loss = np.mean(losses)
        return val_acc, val_loss

    #Для обучения на нескольких эпохах используется метод train,
    #в котором последовательно вызываются методы fit и eval.
    def train(self):
        best_accuracy = 0
        for epoch in range(self.epochs):
            print(f'Epoch {epoch + 1}/{self.epochs}')
            train_acc, train_loss = self.fit()
            print(f'Train loss {train_loss} accuracy {train_acc}')

            val_acc, val_loss = self.eval()
            print(f'Val loss {val_loss} accuracy {val_acc}')
            print('-' * 10)

            if val_acc > best_accuracy:
                torch.save(self.model, self.model_save_path)
                best_accuracy = val_acc

        self.model = torch.load(self.model_save_path)

    # Для предсказания класса для нового текста используется метод predict
    # Метод работает следующим образом:
    # 1 - Токенизируется входной текст;
    # 2 - Токенизированный текст подается в модель;
    # 3 - На выходе получаем вероятности классов;
    # 4 - Возвращаем метку наиболее вероятного класса.
    def predict(self, text):
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            truncation=True,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
        )

        out = {
              'text': text,
              'input_ids': encoding['input_ids'].flatten(),
              'attention_mask': encoding['attention_mask'].flatten()
          }

        input_ids = out["input_ids"].to(self.device)
        attention_mask = out["attention_mask"].to(self.device)

        outputs = self.model(
            input_ids=input_ids.unsqueeze(0),
            attention_mask=attention_mask.unsqueeze(0)
        )

        prediction = torch.argmax(outputs.logits, dim=1).cpu().numpy()[0]

        return prediction

### Инициализация модели:

In [ ]:
classifier = BertClassifier(
        model_path='cointegrated/rubert-tiny2',
        tokenizer_path='cointegrated/rubert-tiny2',
        n_classes=2,
        epochs=2
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Подготовка данных и хэлперов:
Для обучения и оценивания.

In [ ]:
classifier.preparation(
        X_train=list(train['text']),
        y_train=list(train['toxic']),
        X_valid=list(val['text']),
        y_valid=list(val['toxic'])
    )

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


### Обучение модели:

In [ ]:
classifier.train()

## Тестирование модели:

In [ ]:
texts = list(test['text'])
toxic = list(test['toxic'])

predictions = [classifier.predict(t) for t in texts]

In [ ]:
precision, recall, f1score = precision_recall_fscore_support(toxic, predictions,average='binary')[:3]

print(f'precision: {precision}, recall: {recall}, f1score: {f1score}')

## Вывод по моделям:
### Fasttext:
Модель обучилась очень быстро в сравнении с BERT , всего 5 минут. Результат также положительный: f1 метрика равна 0.959

### BERT:
Модель обучается Очень долго в сравнении с Fasttext,
Результат : f1 метрика равна 0.816, что меньше чем у fasttext, но требованиям всё же соответсует.

#### Заказчику я советую использовать модель Fasttext, ткк она быстрее показала лучше результат.

# Вывод по проекту:

## В данном проекте были предприняты следующие шаги:

### 1.) Полученна и анализирована общая информация о данных.

### 2.) Выполнена подготовка данных.

### 3.) Созданны классы для наиболее успешной работы с моделью.

### 4.) Проведена работа с моделями(обучение, тестирование).

### 5.) Сделан Вывод о работе каждой модели.